In [1]:
from jw_package import *

import configuration completed !
train + key + weather merging started
train + key + weather merging finished
function configuration completed ! 
Good to go !


In [4]:
tmp = pd.read_csv('tmp.csv',index_col=0)
X = pd.read_csv('X.csv',index_col=0)
y = pd.read_csv('y.csv',index_col=0)
formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'

X = tmp.loc[:, ['store_nbr', 'item_nbr', 'cool', 'heat', 'preciptotal','snowfall', 'is_holiday']]
y = tmp.loc[:, ['units']]

In [5]:
from sklearn.model_selection import KFold

trial = 4

cv = KFold(n_splits=trial, shuffle=True, random_state=2)

total = 0


for train_index, test_index in cv.split(X):
    print("test index :", test_index, len(test_index))
    print("." * 80)
    print("train index:", train_index, len(train_index))
    print("=" * 80)

    X_train = X.loc[train_index, :]
    X_test = X.loc[test_index, :]
    y_train = y.loc[train_index, :]
    y_test = y.loc[test_index, :]

    model = sm.OLS.from_formula(formula, data=pd.concat([X_train, y_train], axis=1))
    result = model.fit()
    
    y_hat = result.predict(X_test)
    
    ESS = ((y_hat - y_hat.mean())**2).sum()
    TSS = ((y_test-y_test.mean())**2).sum()
    total += ESS/TSS
    print(ESS/TSS)
    
total/trial

test index : [     7     10     14 ..., 118676 118684 118695] 29674
................................................................................
train index: [     0      1      2 ..., 118692 118693 118694] 89022
units    0.689229
dtype: float64
test index : [     0      1      2 ..., 118688 118692 118694] 29674
................................................................................
train index: [     3      5      6 ..., 118691 118693 118695] 89022
units    0.704055
dtype: float64
test index : [     5      6      8 ..., 118686 118689 118693] 29674
................................................................................
train index: [     0      1      2 ..., 118692 118694 118695] 89022
units    0.406551
dtype: float64
test index : [     3     12     15 ..., 118680 118690 118691] 29674
................................................................................
train index: [     0      1      2 ..., 118693 118694 118695] 89022
units    0.703355
dtype: float64


units    0.625798
dtype: float64

In [6]:
res = result.resid_pearson
index = result.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index
len(drop_index)

tmp.drop(list(drop_index),inplace=True)
tmp.reset_index(drop = True, inplace = True)

X = tmp.loc[:, ['store_nbr', 'item_nbr', 'cool', 'heat', 'preciptotal','snowfall', 'is_holiday']]
y = tmp.loc[:, ['units']]